# Import

In [1]:
import csv
import json
import logging
import os
import shutil
import sys
from typing import List
from glob import glob

import boto3
import pandas as pd
from botocore.exceptions import ClientError

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [3]:
from ALL import config 
from util import *
from MultilayerDict import *

In [4]:
s3_bucket_name =  "text-classification-nakayama-bucket"
object_name = "RawData/reuters/"
root_path = "/home/jovyan"

In [5]:
d = {"temporary": {"A": "doc", "B": "tes"}}

In [6]:
file_path = "../../temporary/B/test.csv"
dir_path = "../../temporary/"

In [7]:
f = pd.read_csv("../../temporary/B/test.csv")

In [8]:
encoding="utf8"
newline=""
delimiter=","
quotechar="|"

In [9]:
def _one_dimensional_csv_to_dict(
    file_path, encoding="utf8", newline="", delimiter=",", quotechar="|"
):
    with open(file_path, mode="r", encoding=encoding, newline=newline) as _f:
        _reader = csv.reader(_f, delimiter=delimiter, quotechar=quotechar)
        _dict = dict(_reader)
    return _dict

In [10]:
def _one_dimensional_json_to_dict(
    file_path,
    encoding="utf8",
    newline="",
):
    with open(file_path, mode="r", encoding=encoding, newline=newline) as _f:
        _dict = json.load(_f)
    return _dict

In [33]:
dir_path

'../../temporary/'

In [ ]:
def read_dirs(dir_path, extension="csv", basis_depth="max"):
    # 読み込みディレクトリの中のファイルパスを探索し,ファイルパスからdictのkeyを生成
    _path_dirs = {}
    _abs_dir_path = os.path.abspath(dir_path)
    for _root, _, _file_names in os.walk(_abs_dir_path, followlinks=True):
        if _file_names:
            for _file_name in _file_names:
                _file_path = f"{_root}/{_file_name}"
                _file_name = os.path.splitext(os.path.basename(_file_name))[0]
                _path_dirs[_file_path] = (_file_path.replace(_abs_dir_path, "")).split(
                    "/"
                )[1:]
    # 最も深いパスを基準にMultilayerDictを生成する
    if basis_depth == "max":
        _depth = max(map(len, _path_dirs.values()))
    elif basis_depth == "min":
        _depth = min(map(len, _path_dirs.values()))
    else:
        raise NotImplementedError
    _names = range(_depth)
    _keys = [list(set(_)) for _ in zip(*_path_dirs.values())]
    _md = MultilayerDict(dict(zip(_names, _keys)))
    # ファイルの読み込み
    if extension == "csv":
        for _path, _dir_list in _path_dirs.items():
            md.update(_dir_list, _one_dimensional_csv_to_dict(_path))
    elif extension=="json":
        for _path, _dir_list in _path_dirs.items():
            md.update(_dir_list, _one_dimensional_csv_to_dict(_path))
    else:
        raise NotImplementedError
        
    return True

In [47]:
_path_dirs = {}
_abs_dir_path = os.path.abspath(dir_path)
for _root, _, _file_names in os.walk(_abs_dir_path, followlinks=True):
    if _file_names:
        for _file_name in _file_names:
            _file_path = f"{_root}/{_file_name}"
            _file_name = os.path.splitext(os.path.basename(_file_name))[0]
            _path_dirs[_file_path] = (_file_path.replace(_abs_dir_path, "")).split("/")[1:]

In [48]:
_depth = max(map(len, _path_dirs.values()))
_names = range(_depth)

In [49]:
_keys = [list(set(_)) for _ in zip(*_path_dirs.values())]

In [50]:
md = MultilayerDict(dict(zip(_names, _keys)))

In [51]:
md.dict

{'A': {'test2.csv': {}, 'test.csv': {}},
 'B': {'test2.csv': {}, 'test.csv': {}}}

In [52]:
_path

'/home/jovyan/temporary/B/test'

In [53]:
for _path, _dir_list in _path_dirs.items():
    md.update(_dir_list, one_dimensional_csv_to_dict(_path))

In [54]:
md.dict

{'A': {'test2.csv': {'a': ' 1', 'b': ' 2'},
  'test.csv': {'a': ' 1', 'b': ' 2'}},
 'B': {'test2.csv': {'a': ' 1', 'b': ' 2'},
  'test.csv': {'a': ' 1', 'b': ' 2'}}}